https://github.com/jturkewitz/SideProjects/blob/master/Kaggle/Telstra/telstra.py

In [1]:
import pandas as pd
import numpy as np
import os
import random
import timeit

In [2]:
#tic0 = timeit.default_timer()
tic = timeit.default_timer()
datadirraw = '../data/raw'
datadirprocessed = '../data/processed'
train = pd.read_csv(os.path.join(datadirraw, 'train.csv') , header=0)
test = pd.read_csv(os.path.join(datadirraw, 'test.csv') , header=0)
severity_type = pd.read_csv(os.path.join(datadirraw, 'severity_type.csv'), header=0)
log_feature = pd.read_csv(os.path.join(datadirraw, 'log_feature.csv'), header=0)
event_type = pd.read_csv(os.path.join(datadirprocessed, 'event_type2.csv'), header=0)
resource_type = pd.read_csv(os.path.join(datadirprocessed, 'resource_type2.csv'), header=0)

toc = timeit.default_timer()
print('Load time: ',toc-tic)

Load time:  0.7513363149999996


In [3]:
severity_type['severity_type'].map(lambda x: x.replace(' ','_')).head(4)
severity_type['severity_type'].map(lambda x: x[-1]).head(4)

0    2
1    2
2    2
3    1
Name: severity_type, dtype: object

In [4]:
severity_type['severity_type'] = severity_type['severity_type'].map(lambda x: x.replace(' ','_'))
severity_type['severity_type'] = severity_type['severity_type'].map(lambda x: x[-1])

In [5]:
event_type['event_type'] = event_type['event_type'].map(lambda x: x.replace(' ', '_'))
#event_type['event_type'] = event_type['event_type'].map(lambda x: x[11:]).astype(int)

In [6]:
event_type['event_type'] = event_type['event_type'].map(lambda x: x[11:]).astype(int)

In [7]:
event_type_dummies = pd.get_dummies(event_type,columns=['event_type'])

In [8]:
event_type_dummies.shape

(31170, 54)

In [9]:
resource_type['resource_type'] = resource_type['resource_type'].map(lambda x: x[14:]).astype(int)

In [10]:
resource_type_dummies = pd.get_dummies(resource_type, columns=['resource_type'])

In [11]:
resource_type_dummies.head()

,id,resource_type_1,resource_type_2,resource_type_3,resource_type_4,resource_type_5,resource_type_6,resource_type_7,resource_type_8,resource_type_9,resource_type_10
0,6597,0,0,0,0,0,0,0,1,0,0
1,8011,0,0,0,0,0,0,0,1,0,0
2,2597,0,0,0,0,0,0,0,1,0,0
3,5022,0,0,0,0,0,0,0,1,0,0
4,6852,0,0,0,0,0,0,0,1,0,0


In [12]:
resource_type['resource_type'].head(5)
resource_type['resource_type'].shift(1).head(5)
resource_type['resource_type'].value_counts()

resource_type['resource_shifted_up'] = resource_type['resource_type'].shift(1)
resource_type['resource_shifted_up'].fillna(8, inplace=True)

In [13]:
resource_type['is_next_resource_type_repeat'] = 0
resource_type['is_next_resource_type_different'] = 0

In [14]:
repeat_cond = resource_type['resource_type'] == resource_type['resource_shifted_up']

In [15]:
resource_type['is_next_resource_type_repeat'][repeat_cond] = 1
resource_type['is_next_resource_type_different'][~repeat_cond] = 1

C:\Users\jthe\Anaconda3\envs\spatial\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\jthe\Anaconda3\envs\spatial\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [16]:
resource_type[resource_type['is_next_resource_type_repeat'] == 0]

resource_type[24:27]

,id,resource_type,resource_shifted_up,is_next_resource_type_repeat,is_next_resource_type_different
24,5259,8,8.0,1,0
25,5259,2,8.0,0,1
26,12751,8,2.0,0,1


In [17]:
resource_type['switches_resource_type'] = resource_type['is_next_resource_type_different'].cumsum()

In [18]:
log_feature.head(3)

,id,log_feature,volume
0,6597,feature 68,6
1,8011,feature 68,7
2,2597,feature 68,1


In [19]:
log_feature['log_feature'] = log_feature['log_feature'].map(lambda x: x.replace(' ','_'))
log_feature['log_feature'] = log_feature['log_feature'].map(lambda x: x[8:]).astype(int)
log_feature['log_feature'].dtypes

dtype('int32')

In [20]:
log_feature.index

RangeIndex(start=0, stop=58671, step=1)

In [21]:
log_feature.rename(columns={'index':'count_of_log_feature_seen'},inplace=True)

In [23]:
log_feature.head(3)

,id,log_feature,volume
0,6597,68,6
1,8011,68,7
2,2597,68,1


In [22]:
log_feature_value_counts = log_feature['log_feature'].value_counts().to_dict()

In [24]:
log_feature['num_ids_with_log_feature'] = log_feature['log_feature'].map(lambda x:log_feature_value_counts[x])

In [25]:
log_feature.head(5)

,id,log_feature,volume,num_ids_with_log_feature
0,6597,68,6,1093
1,8011,68,7,1093
2,2597,68,1,1093
3,5022,172,2,214
4,5022,56,1,244


In [26]:
bins = [i for i in range(-10,410,10)]

In [28]:
log_feature['binned_log_feature'] = np.digitize(log_feature['log_feature'], bins, right=True)

In [27]:
log_feature.head(20)

,id,log_feature,volume,num_ids_with_log_feature
0,6597,68,6,1093
1,8011,68,7,1093
2,2597,68,1,1093
3,5022,172,2,214
4,5022,56,1,244
5,5022,193,4,1160
6,5022,71,3,1514
7,6852,201,2,902
8,6852,56,1,244
9,6852,80,2,1336


In [29]:
bins_offset = list(map(lambda x:x+5, bins))

In [30]:
log_feature['binned_offset_log_feature'] = np.digitize(log_feature['log_feature'], bins_offset, right=True)

In [32]:
log_feature['position_of_log_feature'] = 1

In [33]:
log_feature['position_of_log_feature'] = log_feature.groupby(by=['id'])['position_of_log_feature'].cumsum()

In [34]:
log_feature['log_feature'] = log_feature['log_feature'].astype(int)

In [35]:
log_feature[log_feature['id'].isin([4593,14838])].sort_values(by=['id','position_of_log_feature'])

,id,log_feature,volume,num_ids_with_log_feature,binned_log_feature,binned_offset_log_feature,position_of_log_feature
33562,4593,236,5,94,25,25,1
33563,4593,314,73,950,33,32,2
33564,4593,235,1,1294,25,24,3
33565,4593,234,65,882,25,24,4
33566,4593,228,42,438,24,24,5
33567,4593,310,9,399,32,32,6
33568,4593,309,68,627,32,32,7
33569,4593,312,88,5267,33,32,8
33570,4593,315,1,1495,33,32,9
33571,4593,313,11,2145,33,32,10


In [36]:
temp_combined = pd.concat((train, test), axis=0, ignore_index=True)

C:\Users\jthe\Anaconda3\envs\spatial\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [37]:
temp_combined.tail(3)

,fault_severity,id,location
18549,NaN,2374,location 917
18550,NaN,7277,location 208
18551,NaN,9886,location 438


In [38]:
event_type_combined = pd.merge(temp_combined, event_type, how='left', left_on=['id'], right_on=['id'] )

In [39]:
event_type_combined.head(10)

,fault_severity,id,location,event_type
0,1.0,14121,location 118,34
1,1.0,14121,location 118,35
2,0.0,9320,location 91,34
3,0.0,9320,location 91,35
4,1.0,14394,location 152,35
5,1.0,14394,location 152,34
6,1.0,8218,location 931,15
7,1.0,8218,location 931,11
8,0.0,14804,location 120,34
9,0.0,14804,location 120,11


In [40]:
event_type_combined.sort_index()

,fault_severity,id,location,event_type
0,1.0,14121,location 118,34
1,1.0,14121,location 118,35
2,0.0,9320,location 91,34
3,0.0,9320,location 91,35
4,1.0,14394,location 152,35
5,1.0,14394,location 152,34
6,1.0,8218,location 931,15
7,1.0,8218,location 931,11
8,0.0,14804,location 120,34
9,0.0,14804,location 120,11


In [41]:
# #139
most_commonn_event_type_dict = event_type_combined.groupby(['location'])['event_type'].agg(lambda x: x.value_counts().index[0]).to_dict()
#event_type_combined.groupby(['location']).count()

In [42]:
most_commonn_event_type_dict['location 1']

15

In [43]:
event_type_no_dups = event_type_combined.drop_duplicates(['location'])
event_type_no_dups_id = event_type_combined.drop_duplicates(['id'])

In [44]:
print(event_type_no_dups.shape)
print(event_type_no_dups_id.shape)
print(event_type_combined.shape)

(1126, 4)
(18552, 4)
(31170, 4)


In [45]:
event_type_combined.head(3)

,fault_severity,id,location,event_type
0,1.0,14121,location 118,34
1,1.0,14121,location 118,35
2,0.0,9320,location 91,34


In [46]:
event_type_combined['location_number'] = event_type_combined['location'].map(lambda x: x[9:]).astype(int)

In [47]:
event_type_combined.sort_values(['location_number'])

,fault_severity,id,location,event_type,location_number
16854,NaN,7515,location 1,15,1
1553,1.0,3952,location 1,15,1
11182,0.0,2250,location 1,15,1
11183,0.0,2250,location 1,11,1
247,0.0,8011,location 1,15,1
12387,0.0,1849,location 1,15,1
12638,NaN,590,location 1,15,1
12788,NaN,12943,location 1,11,1
12789,NaN,12943,location 1,15,1
1431,0.0,5259,location 1,20,1


In [48]:
print(temp_combined.head(3))
print(log_feature.head(3))
print(event_type_no_dups_id[['id','event_type']].head(3))

   fault_severity     id      location
0             1.0  14121  location 118
1             0.0   9320   location 91
2             1.0  14394  location 152
     id  log_feature  volume  num_ids_with_log_feature  binned_log_feature  \
0  6597           68       6                      1093                   8   
1  8011           68       7                      1093                   8   
2  2597           68       1                      1093                   8   

   binned_offset_log_feature  position_of_log_feature  
0                          8                        1  
1                          8                        1  
2                          8                        1  
      id  event_type
0  14121          34
2   9320          34
4  14394          35


In [49]:
log_combined = pd.merge(temp_combined, log_feature, how='left', left_on='id', right_on='id')
log_combined = pd.merge(log_combined, event_type_no_dups_id[['id', 'event_type']], how='left', left_on='id', right_on='id')

In [50]:
log_combined.head(4)

,fault_severity,id,location,log_feature,volume,num_ids_with_log_feature,binned_log_feature,binned_offset_log_feature,position_of_log_feature,event_type
0,1.0,14121,location 118,312,19,5267,33,32,1,34
1,1.0,14121,location 118,232,19,4754,25,24,2,34
2,0.0,9320,location 91,315,200,1495,33,32,1,34
3,0.0,9320,location 91,235,116,1294,25,24,2,34


In [51]:
# log_feature_unique_features_per_location
log_feature_unique_features_per_location_dict = log_combined.groupby(['location'])['log_feature'].nunique().to_dict()

In [52]:
log_combined[log_combined['location'] == 'location 1']['log_feature'].value_counts()

80     52
82     39
201    33
203    20
56     11
172     7
193     6
68      6
71      5
111     4
198     2
219     1
159     1
134     1
76      1
Name: log_feature, dtype: int64

In [53]:
log_feature_unique_features_per_location_dict['location 1']

15

In [54]:
most_common_event_dict = log_combined.groupby(['location'])['event_type'].agg(lambda x: x.value_counts().index[0]).to_dict()

In [55]:
most_common_feature_dict = log_combined.groupby(['location'])['log_feature'].agg(lambda x: x.value_counts().index[0]).to_dict()

In [56]:
mean_volume_dict = log_combined.groupby(['location'])['volume'].agg(lambda x: x.mean()).to_dict()
max_volume_dict = log_combined.groupby(['location'])['volume'].agg(lambda x: x.max()).to_dict()
std_log_feature_dict = log_combined.groupby(['location'])['log_feature'].agg(lambda x: x.std()).to_dict()

In [57]:
unique_locations_per_feature_dict = log_combined.groupby(['log_feature'])['location'].nunique().to_dict()

In [58]:
median_ids_with_feature_number_dict = log_combined.groupby(['location'])['num_ids_with_log_feature'].agg(lambda x: x.median()).to_dict()
median_feature_number_dict = log_combined.groupby(['location'])['log_feature'].agg(lambda x: x.median()).to_dict()

In [59]:
log_combined['location_number'] = log_combined['location'].map(lambda x:x[9:]).astype(int)

In [60]:
most_common_location_dict = log_combined.groupby(['log_feature'])['location_number'].agg(lambda x: x.value_counts().index[0]).to_dict()

In [61]:
log_combined.sort_index().sort_values(['location_number'])

,fault_severity,id,location,log_feature,volume,num_ids_with_log_feature,binned_log_feature,binned_offset_log_feature,position_of_log_feature,event_type,location_number
29006,NaN,282,location 1,80,3,1336,9,9,1,15,1
11679,0.0,2906,location 1,80,1,1336,9,9,1,15,1
29806,NaN,6693,location 1,80,2,1336,9,9,1,11,1
26422,NaN,4787,location 1,80,3,1336,9,9,1,15,1
45738,NaN,6597,location 1,68,6,1093,8,8,1,11,1
29807,NaN,6693,location 1,82,2,3472,10,9,2,11,1
54723,NaN,10460,location 1,80,5,1336,9,9,2,15,1
54722,NaN,10460,location 1,201,4,902,22,21,1,15,1
25401,NaN,10289,location 1,201,1,902,22,21,2,11,1
25400,NaN,10289,location 1,80,3,1336,9,9,1,11,1


In [62]:
log_feature['locations_per_feature'] = log_feature['log_feature'].map(unique_locations_per_feature_dict)
log_feature['most_common_location_of_log_feature'] = log_feature['log_feature'].map(most_common_location_dict)
log_feature['most_common_event_of_log_feature'] = log_feature['log_feature'].map(most_common_event_dict)

In [64]:
log_feature.head(3)

,id,log_feature,volume,num_ids_with_log_feature,binned_log_feature,binned_offset_log_feature,position_of_log_feature,locations_per_feature,most_common_location_of_log_feature,most_common_event_of_log_feature
0,6597,68,6,1093,8,8,1,225,1026,NaN
1,8011,68,7,1093,8,8,1,225,1026,NaN
2,2597,68,1,1093,8,8,1,225,1026,NaN


In [63]:
unique_locations_per_feature_dict[68]

225

In [70]:
# has_event_type_dict = {}
# event_grouped = event_type_combined.groupby(['location'])
print(event_type_combined[event_type_combined['location'] == 'location 1']['event_type'].value_counts())
print(event_type_combined['event_type'].unique())
event_type_combined.groupby(['location'])['event_type'].unique()

15    66
11    41
20     1
7      1
Name: event_type, dtype: int64
[34 35 15 11 36 20 50 54  2 13 14 44 45 43 42 23 10 22 18  5  3 21 29 40
 39 24 26  7 32 47 46 27 28 49 53  9 38  8 41 30 37  6  1 31 51 12 19 48
 25 52 33 17  4]


location
location 1                     [15, 11, 20, 7]
location 10                               [11]
location 100                  [34, 35, 20, 11]
location 1000                         [20, 11]
location 1001                             [20]
location 1002                             [20]
location 1003                             [20]
location 1004                             [20]
location 1005                             [20]
location 1006                      [11, 20, 9]
location 1007              [11, 15, 54, 2, 20]
location 1008               [15, 11, 1, 6, 20]
location 1009                     [11, 30, 29]
location 101                          [35, 34]
location 1010                         [11, 20]
location 1011                 [20, 11, 30, 29]
location 1012                         [15, 20]
location 1013               [15, 20, 11, 6, 7]
location 1014     [13, 22, 11, 14, 23, 18, 20]
location 1015                         [11, 20]
location 1016                             [11]
loca

In [71]:
has_event_type_dict = {}
event_group = event_type_combined.groupby(['location'])
for i in event_type_combined['event_type'].unique():
    has_event_type_dict[i] = event_group['event_type'].agg(lambda x: 1 if i in x.values else 0).to_dict()

In [72]:
has_event_type_dict[15]

{'location 1': 1,
 'location 10': 0,
 'location 100': 0,
 'location 1000': 0,
 'location 1001': 0,
 'location 1002': 0,
 'location 1003': 0,
 'location 1004': 0,
 'location 1005': 0,
 'location 1006': 0,
 'location 1007': 1,
 'location 1008': 1,
 'location 1009': 0,
 'location 101': 0,
 'location 1010': 0,
 'location 1011': 0,
 'location 1012': 1,
 'location 1013': 1,
 'location 1014': 0,
 'location 1015': 0,
 'location 1016': 0,
 'location 1017': 0,
 'location 1018': 1,
 'location 1019': 1,
 'location 102': 0,
 'location 1020': 0,
 'location 1021': 0,
 'location 1022': 0,
 'location 1023': 0,
 'location 1024': 0,
 'location 1025': 1,
 'location 1026': 0,
 'location 1027': 0,
 'location 1028': 1,
 'location 1029': 0,
 'location 103': 0,
 'location 1030': 0,
 'location 1031': 0,
 'location 1032': 0,
 'location 1033': 0,
 'location 1034': 0,
 'location 1035': 0,
 'location 1036': 1,
 'location 1037': 0,
 'location 1038': 0,
 'location 1039': 0,
 'location 104': 0,
 'location 1040': 0,
 '

In [73]:
event_type_number_dict = event_type_combined.groupby(['location'])['event_type'].agg(lambda x: x.nunique()).to_dict()

In [74]:
event_type_no_dups = event_type_combined.drop_duplicates('id')

In [75]:
print(temp_combined.head(3))
resource_type.head(3)

   fault_severity     id      location
0             1.0  14121  location 118
1             0.0   9320   location 91
2             1.0  14394  location 152


,id,resource_type,resource_shifted_up,is_next_resource_type_repeat,is_next_resource_type_different,switches_resource_type
0,6597,8,8.0,1,0,0
1,8011,8,8.0,1,0,0
2,2597,8,8.0,1,0,0


In [76]:
resource_type_combined = pd.merge(temp_combined, resource_type, how='left', left_on=['id'], right_on=['id'])

In [77]:
most_common_resource_type_dict = resource_type_combined.groupby(['location'])['resource_type'].agg(lambda x: x.value_counts().index[0]).to_dict()
std_resource_type_idct = resource_type_combined.groupby(['location'])['resource_type'].agg(lambda x: x.std()).to_dict()
resource_type_number_dict = resource_type_combined.groupby(['location'])['resource_type'].agg(lambda x: x.nunique()).to_dict()

In [78]:
resource_type.head(3)

,id,resource_type,resource_shifted_up,is_next_resource_type_repeat,is_next_resource_type_different,switches_resource_type
0,6597,8,8.0,1,0,0
1,8011,8,8.0,1,0,0
2,2597,8,8.0,1,0,0


In [82]:
has_resource_type_dict = {}
resource_grouped = resource_type_combined.groupby(['location'])
for i in resource_type_combined.resource_type.unique():
    has_resource_type_dict[i] = resource_grouped['resource_type'].agg(lambda x: 1 if i in x.values else 0).to_dict()

In [83]:
severity_type_combined = pd.merge(temp_combined, severity_type, how='left', left_on='id', right_on='id')

In [89]:
log_feature_dummies = pd.get_dummies(log_feature, columns=['log_feature', 
                                                           'most_common_location_of_log_feature',
                                                           'binned_log_feature', 
                                                           'binned_offset_log_feature'])

In [93]:
#log_feature_dummies_cols = [col for col in list(log_feature_dummies) if col.startswith('log_feature_')]
log_feature_dummies.columns

Index(['id', 'volume', 'num_ids_with_log_feature', 'position_of_log_feature',
       'locations_per_feature', 'most_common_event_of_log_feature',
       'log_feature_1', 'log_feature_2', 'log_feature_3', 'log_feature_4',
       ...
       'binned_offset_log_feature_31', 'binned_offset_log_feature_32',
       'binned_offset_log_feature_33', 'binned_offset_log_feature_34',
       'binned_offset_log_feature_35', 'binned_offset_log_feature_36',
       'binned_offset_log_feature_37', 'binned_offset_log_feature_38',
       'binned_offset_log_feature_39', 'binned_offset_log_feature_40'],
      dtype='object', length=634)

In [92]:
log_feature.columns

Index(['id', 'log_feature', 'volume', 'num_ids_with_log_feature',
       'binned_log_feature', 'binned_offset_log_feature',
       'position_of_log_feature', 'locations_per_feature',
       'most_common_location_of_log_feature',
       'most_common_event_of_log_feature'],
      dtype='object')